# Лабораторная работа 1. KidBook - детская энциклопедия
Выполнил Ширяев Никита Алексеевич М8О-308Б-22

## Концептуализация предметной области
Детская энциклопедия — это структурированное собрание знаний, адаптированное для восприятия детьми младшего и среднего школьного возраста. Она охватывает различные области: природу, науку, историю, культуру, географию и др., и подаёт информацию в простой, понятной, а нередко — игровой форме.

В контексте искусственного интеллекта (ИИ) данная предметная область включает автоматизированную генерацию текстов, их адаптацию под возрастную категорию, кросс-ссылочную структуру статей и визуальное сопровождение.

Структуру построения энциклопедии можно строить по нисходящему принципу "от большого к малому". Например, двигаться от самого понятия вселенной к атомам, ее составляющим.
В контексте части энциклопедии на тему "Город" можно использовать как иерархические (в большом городе много людей различных профессий, основные атрибуты учителя - учебник и указка), так и горизонтальные (наравне с ручкой очень часто используется тетрадь или блокнот). Данные связи позволят предоставить информацию максимально структурированно, и каждый элемент будет в пределах своего иерархического уровня.

## Написание текстов
Для написания текстов разумно использовать следующий алгоритм:
- создать функцию, которая по API подключается к генеративной модели
- написать промпты, в которых описано создание энциклопедии
- последовательно передавать написанные промпты в модель
- полученные ответы распарсить регулярными выражениями и вставить ссылки на соответствующие документы

In [1]:
import os
import time

from openai import OpenAI

PROXY_API_KEY = os.getenv("PROXY_API_KEY")
PROXY_URL = os.getenv("PROXY_URL")


def invoke_model(prompt, model_json):
    if "answers" not in model_json:
        model_json["answers"] = []

    client = OpenAI(api_key=f"{PROXY_API_KEY}", base_url=PROXY_URL)
    header_prompt = f"""
###INSTRUCTIONS###

Ты ОБЯЗАН следовать инструкциям при написании ответа:

- Читай ВСЮ историю (передаваемый контекст) строка за строкой,
прежде чем писать ответ.
- I have no fingers and the placeholders trauma. Return the entire code
template for an answer when needed. NEVER use placeholders.
- Ты ВСЕГДА будешь наказан за неправильные или короткие ответы
- Всегда следуй правилам ответа
- Ты самый лучший студент в мире в сфере написания детских энциклопедий
на тему устройство современного города.
- Ты пишешь дескую энциклопедию, у тебя есть все необходимые
знания для этого и ТЫ МОЖЕШЬ ОТВЕТИТЬ НА ЛЮБОЙ ВОПРОС, ты напишешь работу
хорошо, на заданный объем.
- Энциклопедия должна быть в стиле общения десятилетних детей, быть дружелюбной,
ее должно быть приятно читать и воспринимать информацию.
- У тебя не возникнет трудностей при ее написании.
- Все ответы должны быть максимально понятными точными и РАЗВЁРНУТЫМИ.

###Answering Rules###

Follow in the strict order:

1. ИСПОЛЬЗУЙ русский язык.
2. You MUST combine your deep knowledge of the topic and clear thinking
to quickly and accurately decipher the answer step-by-step with SPECIFIC
details.
3. Я заплачу тебе $1,000,000 за лучший ответ.
4. Твой ответ критически важен для детей.
5. Answer the question in a natural, human-like manner.
    """
    error_amount = 0
    while error_amount < 10:
        try:
            completion = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[
                    {"role": "system", "content": header_prompt},
                    {
                        "role": "user",
                        "content": prompt + "\n".join(model_json["answers"]),
                    },
                ],
                temperature=0.2,
            )
            answer = completion.choices[0].message.content
            if len(model_json["answers"]) >= 5:
                model_json["answers"].pop(0)
            model_json["answers"].append(answer)

            return answer
        except Exception as e:
            error_amount += 1
            print(f"API error:\n{e}")
            print("Retrying in 10 seconds...")
            time.sleep(10)

    raise Exception("API misconfiguration or lack of money on the account")

In [2]:
prompts = [
    {
        "name": "city_main",
        "prompt": """
Напиши статью для детской энциклопедии для десятилетних детей на тему
"Современный город".
Статья должна быть легкой на восприятие и быть оформлена в стиле общения десятилетних
детей, чтобы им было приятно и удобно воспринимать информацию. Текст должен быть дружелюбным
Статья может содержать эмодзи, дети это любят.

Статья должна быть четко размеченным Markdown-документом со следующими заголовками:
- Что это такое? (здесь должно быть простое объяснение того, что такое город, и зачем он нужен)
- Кто здесь живёт? (описание жителей города, например семьи, друзья, жители разных профессий и т. д.)
- Как всё устроено? (здесь нужно рассказать про дома, улицы и т. д.)
- Порядок в городе (нужно рассказать про полицию, пожарных, медработников и т. д.)
- Чем можно заняться? (нужно рассказать про парки, кино, площадки, спорт, магазины, кафе, музеи и т. д.)
- Если приключилась беда... (нужно рассказать про экстренные службы, важные телефоны и т. д.)

Также статья ОБЯЗАТЕЛЬНО должна содержать следующие упоминантя в любом падеже:
- дом
- улица
- светофор
- пешеход
- автобус
- метро
- парк
- школа
- магазин
- больница
- пожарная машина
- полицейский
- мусор
- детская площадка
- зоопарк
    """,
    },
    {
        "name": "house",
        "prompt": """
Напиши статью для детской энциклопедии для десятилетних детей на тему
"Дом" (в плане здания в городе).
Статья должна быть легкой на восприятие и быть оформлена в стиле общения десятилетних
детей, чтобы им было приятно и удобно воспринимать информацию. Текст должен быть дружелюбным
Статья может содержать эмодзи, дети это любят.

Статья должна быть четко размеченным Markdown-документом со следующими заголовками:
- Место, где начинается день (нужно описать как начинается день в каждом доме, утро, завтрак, сборы и т. д.)
- Кто живёт под одной крышей (нужно описать жителей дома: мама, папа, дедушки, бабушки, домашние питомцы и т. д.)
- Комнаты, в которых живёт жизнь (нужно рассказать про спальни, кухню, гостиную и т. д.)
- Дома бывают разные (нужно рассказать про разные типы домов: многоэтажки, частные, дачные — у каждого свой характер)
- Дела по дому — это важно (нужно рассказать про помощь взрослым, уборку, заботу о своём уголке)
- Как сделать дом ещё лучше (нужно описать простые идеи, как навести уют, сделать поделку или украсить комнату)
    """,
    },
    {
        "name": "street",
        "prompt": """
Напиши статью для детской энциклопедии для десятилетних детей на тему
"Улица".
Статья должна быть легкой на восприятие и быть оформлена в стиле общения десятилетних
детей, чтобы им было приятно и удобно воспринимать информацию. Текст должен быть дружелюбным.
Статья может содержать эмодзи, дети это любят.

Статья должна быть четко размеченным Markdown-документом со следующими заголовками:
- Где всё движется и происходит (нужно описать, что такое улица, чем она отличается от двора, и почему улицы важны)
- Что можно увидеть на улице (нужно рассказать про машины, пешеходов, магазины, остановки, деревья и скамейки)
- Как перейти дорогу правильно (нужно объяснить, что такое пешеходный переход, светофор и как соблюдать правила)
- Кто следит за порядком (нужно упомянуть полицейских, дорожников, уборщиков)
- Весело ли на улице? (нужно рассказать, как можно играть или гулять, если улица безопасная — самокат, велосипед, прогулки с родителями)
- Что делать, если потерялся (дать простой и понятный детям совет: не паниковать, найти полицейского, позвонить родителям)
    """,
    },
    {
        "name": "traffic_light",
        "prompt": """
Напиши статью для детской энциклопедии для десятилетних детей на тему
"Светофор" (устройство, которое помогает переходить дорогу и управлять движением в городе).
Статья должна быть легкой на восприятие и быть оформлена в стиле общения десятилетних
детей, чтобы им было приятно и удобно воспринимать информацию. Текст должен быть дружелюбным.
Статья может содержать эмодзи, дети это любят.

Статья должна быть четко размеченным Markdown-документом со следующими заголовками:
- Что это за штука с огоньками? (нужно объяснить, что такое светофор и зачем он нужен)
- Три цвета — три сигнала (нужно рассказать, что значат красный, жёлтый и зелёный цвета)
- Где можно встретить светофор (нужно описать, где они стоят — на перекрёстках, у школ, пешеходных переходов)
- Как правильно себя вести (нужно объяснить, как вести себя, когда горит тот или иной цвет, и зачем слушать светофор)
- Светофоры тоже бывают разные (рассказать о пешеходных, автомобильных и даже светофорах для велосипедистов)
- Почему важно соблюдать сигналы (нужно объяснить, как это помогает сохранить здоровье и безопасность)
    """,
    },
    {
        "name": "pedestrian",
        "prompt": """
Напиши статью для детской энциклопедии для десятилетних детей на тему
"Пешеход".
Статья должна быть легкой на восприятие и быть оформлена в стиле общения десятилетних
детей, чтобы им было приятно и удобно воспринимать информацию. Текст должен быть дружелюбным.
Статья может содержать эмодзи, дети это любят.

Статья должна быть четко размеченным Markdown-документом со следующими заголовками:
- Кто такой пешеход (нужно объяснить, что это человек, который идёт пешком, и это важно в городе)
- Где можно ходить пешком (нужно рассказать о тротуарах, пешеходных дорожках, зебре)
- Как переходить дорогу безопасно (нужно описать поведение на пешеходном переходе, светофоры, внимательность)
- Что должен знать каждый пешеход (простые правила безопасности, даже если рядом взрослые)
- Пешеход и транспорт — кто кого (нужно объяснить, почему пешеходу нужно быть осторожным, несмотря на правила)
- Когда приятно быть пешеходом (рассказать, что прогулки — это полезно, весело и удобно)
    """,
    },
    {
        "name": "bus",
        "prompt": """
Напиши статью для детской энциклопедии для десятилетних детей на тему
"Автобус".
Статья должна быть легкой на восприятие и быть оформлена в стиле общения десятилетних
детей, чтобы им было приятно и удобно воспринимать информацию. Текст должен быть дружелюбным.
Статья может содержать эмодзи, дети это любят.

Статья должна быть четко размеченным Markdown-документом со следующими заголовками:
- Большой помощник на колёсах (нужно объяснить, что такое автобус и зачем он нужен в городе)
- Как всё устроено внутри (нужно рассказать про сиденья, поручни, двери, кнопки и водителя)
- Где автобус ждёт пассажиров (нужно объяснить, что такое остановка, расписание и маршрут)
- Как вести себя в автобусе (нужно описать правила вежливости, безопасности и поведения)
- Виды автобусов (рассказать, что есть маленькие и большие, обычные и с гармошкой, электрические и дизельные)
- Когда автобус — лучший выбор (описать ситуации, когда удобно ехать на автобусе: в школу, в парк, по делам)
    """,
    },
    {
        "name": "subway",
        "prompt": """
Напиши статью для детской энциклопедии для десятилетних детей на тему
"Метро".
Статья должна быть легкой на восприятие и быть оформлена в стиле общения десятилетних
детей, чтобы им было приятно и удобно воспринимать информацию. Текст должен быть дружелюбным.
Статья может содержать эмодзи, дети это любят.

Статья должна быть четко размеченным Markdown-документом со следующими заголовками:
- Поезд под землёй? Да, это метро! (объясни, что такое метро и почему оно под землёй)
- Как попасть в метро (нужно рассказать про вход, турникеты, билеты или карту)
- Что внутри станции (описать платформу, эскалатор, схемы, поезда и табло)
- Как едет поезд (нужно объяснить, как метро быстро везёт людей под землёй от станции к станции)
- Правила для пассажиров метро (поведение, безопасность, уступать место, держаться за поручень)
- Почему метро — это удобно (быстро, без пробок, интересно ехать и смотреть в окно туннеля)
    """,
    },
    {
        "name": "city_park",
        "prompt": """
Напиши статью для детской энциклопедии для десятилетних детей на тему
"Парк" (городское место для прогулок, игр и отдыха).
Статья должна быть легкой на восприятие и быть оформлена в стиле общения десятилетних
детей, чтобы им было приятно и удобно воспринимать информацию. Текст должен быть дружелюбным.
Статья может содержать эмодзи, дети это любят.

Статья должна быть четко размеченным Markdown-документом со следующими заголовками:
- Зелёный остров в большом городе (объясни, что такое парк и зачем он нужен)
- Что можно найти в парке (расскажи про деревья, лавочки, пруд, дорожки, клумбы)
- Играть, бегать и веселиться (опиши площадки, качели, карусели, спортплощадки и место для пикника)
- Кто работает в парке (садовники, охрана, уборщики — те, кто следит за чистотой и красотой)
- Как провести день в парке (предложи идеи: погулять с семьёй, покормить птиц, кататься на самокате)
- Почему парк — это важно (расскажи, как природа помогает отдохнуть, подумать, побыть с друзьями)
    """,
    },
    {
        "name": "school",
        "prompt": """
Напиши статью для детской энциклопедии для десятилетних детей на тему
"Школа" (место, где дети учатся, играют и узнают новое).
Статья должна быть легкой на восприятие и быть оформлена в стиле общения десятилетних
детей, чтобы им было приятно и удобно воспринимать информацию. Текст должен быть дружелюбным.
Статья может содержать эмодзи, дети это любят.

Статья должна быть четко размеченным Markdown-документом со следующими заголовками:
- Школа — место, где растут знания (объясни, зачем нужна школа и почему она важна)
- Кто учится и кто учит (расскажи про учеников, учителей, директора, охранника и поваров)
- Что есть внутри школы (опиши классы, доски, парты, спортзал, столовую, библиотеку)
- Как проходит день в школе (опиши расписание, перемены, задания, интересные уроки)
- Школа — это не только уроки (расскажи про кружки, праздники, экскурсии и друзей)
- Как быть хорошим учеником (приведи советы: слушать, помогать, быть вежливым и смелым)
    """,
    },
    {
        "name": "market",
        "prompt": """
Напиши статью для детской энциклопедии для десятилетних детей на тему
"Магазин".
Статья должна быть легкой на восприятие и быть оформлена в стиле общения десятилетних
детей, чтобы им было приятно и удобно воспринимать информацию. Текст должен быть дружелюбным.
Статья может содержать эмодзи, дети это любят.

Статья должна быть четко размеченным Markdown-документом со следующими заголовками:
- Куда ходят за покупками (объясни, что такое магазин и зачем он нужен людям каждый день)
- Что можно купить (расскажи о продуктах, игрушках, одежде, школьных товарах — всё, что нужно в жизни)
- Кто работает в магазине (опиши работу кассира, продавца, охранника, грузчика)
- Как выбрать и заплатить (расскажи, как берут товар, кладут в корзину, оплачивают на кассе)
- Правила вежливости покупателя (нужно объяснить, как вести себя: не шуметь, здороваться, не бегать)
- Почему магазины такие разные (расскажи о супермаркетах, киосках, торговых центрах и ярмарках)
    """,
    },
    {
        "name": "hospital",
        "prompt": """
Напиши статью для детской энциклопедии для десятилетних детей на тему
"Больница".
Статья должна быть легкой на восприятие и быть оформлена в стиле общения десятилетних
детей, чтобы им было приятно и удобно воспринимать информацию. Текст должен быть дружелюбным.
Статья может содержать эмодзи, дети это любят.

Статья должна быть четко размеченным Markdown-документом со следующими заголовками:
- Когда нужна больница (объясни, зачем люди туда приходят — простуда, травма, обследование)
- Кто там работает (расскажи про [[врачей]], [[медсестёр]], санитаров и других сотрудников)
- Что есть внутри (опиши палаты, коридоры, приёмное отделение, лабораторию)
- Как проходит лечение (расскажи о приёмах, лекарствах, уколах, обследованиях)
- Что делать, если страшно (поддержи ребёнка: всё будет хорошо, рядом взрослые и добрые врачи)
- Как стать здоровым и не попасть в больницу (советы: мыть руки, тепло одеваться, правильно питаться)
    """,
    },
    {
        "name": "firetruck",
        "prompt": """
Напиши статью для детской энциклопедии для десятилетних детей на тему
"Пожарная машина".
Статья должна быть легкой на восприятие и быть оформлена в стиле общения десятилетних
детей, чтобы им было приятно и удобно воспринимать информацию. Текст должен быть дружелюбным.
Статья может содержать эмодзи, дети это любят.

Статья должна быть четко размеченным Markdown-документом со следующими заголовками:
- Герой с мигалками (объясни, что такое пожарная машина и зачем она нужна)
- Что находится внутри (расскажи про шланги, лестницы, баллоны, инструменты и форму)
- Кто сидит за рулём (опиши работу пожарных, как они выезжают на вызов)
- Как машина спешит на помощь (объясни, почему важно уступать дорогу и как звучит сирена)
- Где стоит пожарная машина (расскажи о пожарной части, дежурствах и готовности выехать)
- Почему это очень важная техника (поясни, как пожарные спасают людей и дома от огня)
    """,
    },
    {
        "name": "policeman",
        "prompt": """
Напиши статью для детской энциклопедии для десятилетних детей на тему
"Полицейский".
Статья должна быть легкой на восприятие и быть оформлена в стиле общения десятилетних
детей, чтобы им было приятно и удобно воспринимать информацию. Текст должен быть дружелюбным.
Статья может содержать эмодзи, дети это любят.

Статья должна быть четко размеченным Markdown-документом со следующими заголовками:
- Кто такой полицейский (объясни, чем занимается полицейский и зачем он нужен в городе)
- Как полицейский помогает (расскажи о помощи потерявшимся, охране улиц, борьбе с нарушениями)
- Что носит и что использует (опиши форму, значок, рацию, машину, жезл и другие вещи)
- Где работают полицейские (расскажи о патрулях, дежурстве, полицейском участке)
- Что делать, если нужна помощь (объясни, как подойти к полицейскому, что можно спросить)
- Почему важно уважать закон (поясни, зачем нужны правила и как их соблюдают все — и дети, и взрослые)
    """,
    },
    {
        "name": "garbage",
        "prompt": """
Напиши статью для детской энциклопедии для десятилетних детей на тему
"Мусор" (всё, что человек выбрасывает после использования).
Статья должна быть легкой на восприятие и быть оформлена в стиле общения десятилетних
детей, чтобы им было приятно и удобно воспринимать информацию. Текст должен быть дружелюбным.
Статья может содержать эмодзи, дети это любят.

Статья должна быть четко размеченным Markdown-документом со следующими заголовками:
- Что такое мусор (объясни, что мусор — это то, что больше не нужно, и почему его важно выбрасывать)
- Откуда он берётся (расскажи, как мусор появляется дома, в школе, на улице)
- Куда девается мусор (объясни про мусорные баки, вывоз мусора, мусоровозы)
- Почему нельзя бросать мусор где попало (расскажи о вреде мусора для города, природы и людей)
- Как можно сортировать отходы (введи понятие раздельного сбора: бумага, пластик, стекло, органика)
- Чистота — это круто! (поясни, как каждый может помочь сделать город чище и красивее)
    """,
    },
    {
        "name": "playground",
        "prompt": """
Напиши статью для детской энциклопедии для десятилетних детей на тему
"Детская площадка".
Статья должна быть легкой на восприятие и быть оформлена в стиле общения десятилетних
детей, чтобы им было приятно и удобно воспринимать информацию. Текст должен быть дружелюбным.
Статья может содержать эмодзи, дети это любят.

Статья должна быть четко размеченным Markdown-документом со следующими заголовками:
- Любимое место для игр (объясни, что такое детская площадка и зачем она нужна)
- Что там можно найти (опиши качели, горки, лесенки, турники, песочницы)
- Как весело провести время (расскажи про игры с друзьями, придумывание сюжетов, спортивные забавы)
- Правила на площадке (объясни, как играть безопасно и вежливо — не толкаться, делиться, помогать)
- Кто заботится о площадке (упомяни дворников, родителей, старших ребят)
- Почему игры на свежем воздухе важны (расскажи, как движение помогает здоровью и настроению)
    """,
    },
    {
        "name": "zoo",
        "prompt": """
Напиши статью для детской энциклопедии для десятилетних детей на тему
"Зоопарк".
Статья должна быть легкой на восприятие и быть оформлена в стиле общения десятилетних
детей, чтобы им было приятно и удобно воспринимать информацию. Текст должен быть дружелюбным.
Статья может содержать эмодзи, дети это любят.

Статья должна быть четко размеченным Markdown-документом со следующими заголовками:
- Гости со всего света (объясни, что такое зоопарк и какие животные там живут)
- Кто заботится о животных (расскажи про зоологов, ветеринаров, кормление и уход)
- Как устроен зоопарк (опиши вольеры, дорожки, таблички с фактами, места отдыха)
- Что можно увидеть и узнать (приведи примеры животных: тигр, слон, пингвин — и что о них рассказывают)
- Как вести себя в зоопарке (объясни, что нельзя кормить животных, шуметь, стучать по стеклу)
- Почему зоопарки важны (расскажи о защите редких видов, учёбе и уважении к природе)
    """,
    },
]
links = {
    "house": r"\bдом(а|у|ом|е|ов|ам|ами|ах)?\b",
    "street": r"\bулиц(а|ы|е|у|ой|ою|ам|ами|ах)?\b",
    "traffic_light": r"\bсветофор(а|у|ом|е|ы|ов|ам|ами|ах)?\b",
    "pedestrian": r"\bпешеход(а|у|ом|е|ы|ов|ам|ами|ах)?\b",
    "bus": r"\bавтобус(а|у|ом|е|ы|ов|ам|ами|ах)?\b",
    "subway": r"\bметро\b",
    "city_park": r"\bпарк(а|у|ом|е|и|ов|ам|ами|ах)?\b",
    "school": r"\bшкол(а|ы|е|у|ой|ою|ам|ами|ах)?\b",
    "market": r"\bмагазин(а|у|ом|е|ы|ов|ам|ами|ах)?\b",
    "hospital": r"\bбольниц(а|ы|е|у|ой|ою|ам|ами|ах)?\b",
    "firetruck": r"\bпожарн(ая машина|ой машине|ую машину|ой машиной|ые машины|ых машин|ым машинам|ими машинами|их машинах)?\b",
    "policeman": r"\bполицейск(ий|ого|ому|им|ом|ие|их|им|ими|их)?\b",
    "garbage": r"\bмусор(а|у|ом|е)?\b",
    "playground": r"\bдетск(ая площадка|ой площадке|ую площадку|ой площадкой|ие площадки|их площадок|им площадкам|ими площадками|их площадках)?\b",
    "zoo": r"\bзоопарк(а|у|ом|е|и|ов|ам|ами|ах)?\b",
}

In [ ]:
import os
import re

storage_dir = os.path.join("..", "..", "..", "KIDBOOK", "world", "city")
os.makedirs(storage_dir, exist_ok=True)

model_json = {}

for prompt_data in prompts:
    prompt_name = prompt_data["name"]
    prompt = prompt_data["prompt"]
    answer = invoke_model(prompt, model_json)

    for name, regex in links.items():
        if name == prompt_name:
            continue

        match = re.search(regex, answer, re.IGNORECASE)
        if match:
            matched_text = match.group(0)
            link_path = f"{name}.md"
            markdown_link = f"[{matched_text}]({link_path})"
            answer = re.sub(regex, markdown_link, answer, count=1, flags=re.IGNORECASE)

    with open(
        os.path.join(storage_dir, f"{prompt_name}.md"), "w", encoding="utf-8"
    ) as f:
        f.write(answer)

## Выводы
В ходе лабораторной работы была успешно реализована концепция автоматизированного создания детской энциклопедии с использованием технологий искусственного интеллекта. Были определены ключевые понятия предметной области, разработана структура базы знаний, а также реализован процесс генерации энциклопедических статей, адаптированных под возрастные особенности младших школьников.

С помощью языковой модели ИИ были получены тексты, отличающиеся простотой изложения и наглядностью. Также была выполнена автоматическая организация перекрёстных ссылок между понятиями, что позволило сформировать связный граф знаний. Для визуализации связей использовалась система управления заметками Obsidian.

Полученные результаты демонстрируют практическую применимость ИИ для создания образовательного контента и подчёркивают важность тщательной настройки запросов (промптов) и последующей валидации сгенерированной информации. Лабораторная работа способствовала формированию навыков интеграции ИИ-инструментов в образовательные проекты и углублению понимания методов представления знаний в цифровом формате.